In [47]:
import json

def detect_lang(text):
    by_letters = ['і', 'ў', 'шч']
    for by_letter in by_letters:
        if by_letter in text:
            return "by"
    return "ru"

In [38]:
# Load events fetched from tutby, relax, citydog

with open("C:\\Projects\\Research\\Events\\data\\event_data\\raw_data\\2020_5_8_0\\events_reduced.json", "r+", encoding="utf-8") as f:
    events = json.load(f)

In [39]:
events[0]

{'title': 'Концерт Руслана Алехно и ансамбля Вооруженных сил РБ с программой "Наследие"',
 'short_description': '',
 'description': 'О событии\n«Наследие» - это специально подготовленная программа Руслана Алехно и\n«Заслуженного коллектива Республики Беларусь «Академического ансамбля песни и танца Вооруженных Сил Республики Беларусь» под руководством Дмитрия Наумовича.\nВ праздничном концерте, посвященном 75-летию Победы в Великой Отечественной войне прозвучат знаменитые песни военных лет и любимые мелодии военной поры.\xa0\nСтоимость билетов:\xa020 - 40 руб.',
 'poster': 'https://img.afisha.tut.by/static/media/340x0s/cover/0c/2/koncert-ruslana-alekhno-i-ansamblya-vooruzhennykh-sil-rb-s-programmoy-nasledie-408465.jpg',
 'url': 'https://afisha.tut.by/concert/ruslan_alekhno/',
 'place': {'name': 'Центральный Дом офицеров',
  'address': 'г. Минск, ул. Красноармейская, 3',
  'url': 'https://afisha.tut.by/place/dom-ofitserov/'},
 'tags': ['Концерт', 'Поп'],
 'age': None,
 'registration': No

In [51]:
# Load tags and types

with open("C:\\Projects\\Research\\Events\\data\\tags\\unified_tags.json", "r+", encoding="utf-8") as f:
    tags_types = json.load(f)
    
event_types = tags_types['types']
event_tags = tags_types['tags']

In [52]:
new_type_mapping = {
    "Форум": "Конференция",
    "Воркшоп": "Мастер-класс",
    "Турнир": "Спортивное мероприятие",
    "Кино": "Просмотр",
    "Творческая встреча": "Встреча",
    "Митап": "Семинар",
    "Занятие": "Мастер-класс",
    "Литературный вечер": "Семинар"
}

In [43]:
event_types[0]

'Автограф-сессия'

In [50]:
event_tags['sport']

TypeError: list indices must be integers or slices, not str

In [48]:
# Create file for type classification
# {"text": "Terrible customer service.", "labels": ["negative"]}

with open("C:\\Projects\\Research\\Events\\data\\doccano\\type_classification.jsonl", "w+", encoding="utf-8") as f:
    for event in events:
        event_description = event['title'].strip() + "\n" + event['description'].strip()
        if len(event_description) < 50:
            continue
        if detect_lang(event_description) == "by":
            continue
        event_tags = []
        for t in event['tags']:
            if t in new_type_mapping:
                event_tags.append(new_type_mapping[t])
            else:
                event_tags.append(t)
                
        labels = [t for t in event_tags if t in event_types]
        if len(labels) == 0:
            labels = ["-"]
        labeled_event = {"text": event_description, "labels": labels}
        f.write(json.dumps(labeled_event, ensure_ascii=False) + "\n")

In [53]:
# Create file for theme classification
# {"text": "Terrible customer service.", "labels": ["negative"]}

# create reversed index
tag_to_theme = {}
for group, tags in event_tags.items():
    for tag in tags:
        tag_to_theme[tag] = group



with open("C:\\Projects\\Research\\Events\\data\\doccano\\theme_classification", "w+", encoding="utf-8") as f:
    for event in events:
        event_description = event['title'].strip() + "\n" + event['description'].strip()
        if len(event_description) < 50:
            continue
        if detect_lang(event_description) == "by":
            continue
        labels = [t for t in event['tags'] if t in tag_to_theme]
        labeled_event = {"text": event_description, "labels": [tag_to_theme[l] for l in labels]}
        f.write(json.dumps(labeled_event, ensure_ascii=False) + "\n")